In [2]:
import pandas as pd

# Load the dataset (assuming the file is in the same directory as the notebook)
file_name = "Heart_Disease_Classification_Dataset.csv"
data = pd.read_csv(file_name)

In [3]:
print("Dataset Preview:")
print(data.head())
print("\nBasic Report:")
print(data.describe())

Dataset Preview:
   Unnamed: 0  age     sex  cp  trestbps   chol  fbs  restecg  thalach  exang  \
0           0   63    male   3     145.0  233.0    1        0    150.0      0   
1           1   37    male   2     130.0  250.0    0        1    187.0      0   
2           2   41  female   1     130.0  204.0    0        0    172.0      0   
3           3   56    male   1     120.0  236.0    0        1    178.0      0   
4           4   57  female   0       NaN  354.0    0        1    163.0      1   

   oldpeak  slope  ca  thal target  
0      2.3      0   0     1    yes  
1      3.5      0   0     2    yes  
2      1.4      2   0     2    yes  
3      0.8      2   0     2    yes  
4      0.6      2   0     2    yes  

Basic Report:
       Unnamed: 0         age          cp    trestbps        chol         fbs  \
count  303.000000  303.000000  303.000000  299.000000  302.000000  303.000000   
mean   151.000000   54.366337    0.966997  131.712375  246.317881    0.148515   
std     87.61278

In [4]:
# Check for missing values in each column
print("\nMissing Values by Column:")
missing_values = data.isnull().sum()
print(missing_values[missing_values > 0])


Missing Values by Column:
trestbps    4
chol        1
thalach     5
dtype: int64
